In [12]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
fileNameQrd = 'qrd_canonical_model.csv'
fileNameMatchRuleBook = 'ruleDict.json'
fileNameDocumentTypeNames = 'documentTypeNames.json'

fsMountName = '/mounted'

In [14]:
import pprint
import pandas as pd
import uuid
import json
import os
import glob
import re
import sys
from bs4 import NavigableString, BeautifulSoup
from collections import defaultdict
import random
import string

from utils.config import config
from utils.logger.logger import loggerCreator

# ePI Modules
from parse.rulebook.rulebook import StyleRulesDictionary

from parse.extractor.parser import parserExtractor
from match.matchDocument.matchDocument import MatchDocument
from documentAnnotation.documentAnnotation import DocumentAnnotation
from htmlDocTypePartitioner.partition import DocTypePartitioner
from extractContentBetweenHeadings.dataBetweenHeadingsExtractor import DataBetweenHeadingsExtractor
from fhirXmlGenerator.fhirXmlGenerator import FhirXmlGenerator
from fhirService.fhirService import FhirService
from utils.logger.matchLogger import MatchLogger
from languageInfo.documentTypeNames.documentTypeNames import DocumentTypeNames


class FolderNotFoundError(Exception):
    pass


def getRandomString(N):
    str_ = ''.join(random.choice(string.ascii_uppercase + string.digits
                                 + string.ascii_lowercase) for _ in range(N))
    return str_


def convertHtmlToJson(domain, procedureType, languageCode, fileNameHtml, fileNameQrd, fileNameLog, fsMountName, localEnv):

    if localEnv is True:
        pathSep = "\\"
        module_path = os.path.join(os.path.abspath(os.path.join('..')), 'data', 'converted_to_html', f'{domain}', f'{procedureType}', f'{languageCode}')
    else:
        pathSep = "/"
        module_path = os.path.join(f'{fsMountName}', 'data', 'converted_to_html', f'{domain}', f'{procedureType}', f'{languageCode}')
    
    # Generate output folder path
    output_json_path = module_path.replace('converted_to_html', 'outputJSON')

    """
        Check if input folder exists, else throw exception
    """
    if(os.path.exists(module_path)):
        filenames = glob.glob(os.path.join(module_path, fileNameHtml))

        # Create language specific folder in outputJSON folder if it doesn't exist
        if(not os.path.exists(output_json_path)):
            os.mkdir(output_json_path)
        logger = MatchLogger(f'Parser_{getRandomString(1)}', fileNameHtml,
                             domain, procedureType, languageCode, "HTML", fileNameLog)

        styleLogger = MatchLogger(
            f'Style Dictionary_{getRandomString(1)}', fileNameHtml, domain, procedureType, languageCode, "HTML", fileNameLog)

        styleRulesObj = StyleRulesDictionary(styleLogger,
                                             language=languageCode,
                                             fileName=fileNameQrd,
                                             domain=domain,
                                             procedureType=procedureType,
                                             fsMountName=fsMountName,
                                             localEnv=localEnv)

        parserObj = parserExtractor(config, logger, styleRulesObj.styleRuleDict,
                                    styleRulesObj.styleFeatureKeyList,
                                    styleRulesObj.qrd_section_headings)

        for input_filename in filenames:
          # if(input_filename.find('Kalydeco II-86-PI-clean')!=-1):
            output_filename = input_filename.replace('converted_to_html', 'outputJSON')
            style_filepath =  output_filename.replace('.html','.txt')
            style_filepath =  style_filepath.replace('.txtl','.txt')
            style_filepath =  style_filepath.replace('.htm','.txt')
            print("-------------",style_filepath,"-----------------")

            output_filename = output_filename.replace('.html', '.json')
            output_filename = output_filename.replace('.htm', '.json')
            print(input_filename, output_filename)
            parserObj.createPIJsonFromHTML(input_filepath=input_filename,
                                           output_filepath=output_filename,
                                           style_filepath = style_filepath,
                                           img_base64_dict=parserObj.convertImgToBase64(input_filename)
                                           )
            
        return output_filename.split(pathSep)[-1], style_filepath
    else:
        try:    
            raise FolderNotFoundError(module_path + " not found")
        except:  
            logger.logFlowCheckpoint("Folder For Language Code Not Found In Input File")
            logger.logException("Folder For Language Code Not Found In Input File")
        raise FolderNotFoundError(module_path + " not found")
        return None


def splitJson(domain, procedureType, languageCode, fileNameJson, fileNameQrd, fileNameLog, fsMountName, localEnv):

    styleLogger = MatchLogger(
        f'Style Dictionary_{getRandomString(1)}', fileNameJson, domain, procedureType, languageCode, "Json", fileNameLog)

    styleRulesObj = StyleRulesDictionary(styleLogger,
                                             language=languageCode,
                                             fileName=fileNameQrd,
                                             domain=domain,
                                             procedureType=procedureType,
                                             fsMountName=fsMountName,
                                             localEnv=localEnv)
    if localEnv is True:
        path_json = os.path.join(os.path.abspath(os.path.join(
                '..')), 'data', 'outputJSON', domain, procedureType, languageCode, fileNameJson)
    else:
        path_json = os.path.join(f'{fsMountName}', 'data', 'outputJSON', domain,  procedureType, languageCode, fileNameJson)
        
    partitionLogger = MatchLogger(
        f'Partition_{getRandomString(1)}', fileNameJson, domain, procedureType, languageCode, "Json", fileNameLog)

    partitioner = DocTypePartitioner(partitionLogger, localEnv)

    partitionedJsonPaths = partitioner.partitionHtmls(
        styleRulesObj.qrd_section_headings, path_json)

    return partitionedJsonPaths


def extractAndValidateHeadings(domain,
                               procedureType,
                               languageCode,
                               documentNumber,
                               fileNameDoc,
                               fileNameQrd,
                               fileNameMatchRuleBook,
                               fileNameDocumentTypeNames,
                               fileNameLog,
                               stopWordFilterLen=6,
                               isPackageLeaflet=False,
                               medName=None,
                               fsMountName='/mounted',
                               localEnv=False):

    if documentNumber == 0:
        topHeadingsConsidered = 4
        bottomHeadingsConsidered = 6
    elif documentNumber == 1:
        topHeadingsConsidered = 3
        bottomHeadingsConsidered = 5
    elif documentNumber == 2:
        topHeadingsConsidered = 5
        bottomHeadingsConsidered = 15
    else:
        topHeadingsConsidered = 5
        bottomHeadingsConsidered = 10

    print(f"Starting Heading Extraction For File :- {fileNameDoc}")
    logger = MatchLogger(f"Heading Extraction {fileNameDoc}_{getRandomString(1)}", fileNameDoc, domain, procedureType, languageCode, documentNumber, fileNameLog)
    logger.logFlowCheckpoint("Starting Heading Extraction")

    stopWordlanguage = DocumentTypeNames(
        fileNameDocumentTypeNames=fileNameDocumentTypeNames,
        languageCode=languageCode,
        domain=domain,
        procedureType=procedureType,
        documentNumber=documentNumber,
        fsMountName=fsMountName,
        localEnv=localEnv).extractStopWordLanguage()

    matchDocObj = MatchDocument(
        logger,
        domain,
        procedureType,
        languageCode,
        documentNumber,
        fileNameDoc,
        fileNameQrd,
        fileNameMatchRuleBook,
        fileNameDocumentTypeNames,
        topHeadingsConsidered,
        bottomHeadingsConsidered,
        stopWordFilterLen,
        stopWordlanguage,
        isPackageLeaflet,
        medName,
        fsMountName,
        localEnv)
    df, coll = matchDocObj.matchHtmlHeaddingsWithQrd()

    return df, coll


def parseDocument(htmlDocPath, fileNameQrd, fileNameMatchRuleBook, fileNameDocumentTypeNames, fsMountName = '/mounted', localEnv= False, medName = None):
    
    if localEnv is True:
        pathSep = "\\"
        fileNameLog = os.path.join(os.path.abspath(os.path.join('..')),'data','FinalLog.txt')
    else:
        pathSep = "/"
        fileNameLog = os.path.join(f'{fsMountName}','data','FinalLog.txt')
    pathComponents = htmlDocPath.split(pathSep)
    print(pathComponents)
    fileNameHtml = pathComponents[-1]
    languageCode =  pathComponents[-2]
    procedureType = pathComponents[-3]
    domain = pathComponents[-4]

    #procedureType = "CAP"
    print(fileNameHtml,languageCode,procedureType)
    
    
    
    flowLogger =  MatchLogger(f"Flow Logger HTML_{getRandomString(1)}", fileNameHtml, domain, procedureType, languageCode, "HTML", fileNameLog)

    flowLogger.logFlowCheckpoint("Starting HTML Conversion To Json")
    ###Convert Html to Json
    fileNameJson, stylesFilePath = convertHtmlToJson( domain, procedureType, languageCode, fileNameHtml, fileNameQrd, fileNameLog, fsMountName, localEnv)
    
    print(stylesFilePath)
    flowLogger.logFlowCheckpoint("Completed HTML Conversion To Json")

    flowLogger.logFlowCheckpoint("Starting Json Split")

    ###Split Uber Json to multiple Jsons for each category.
    partitionedJsonPaths = splitJson(domain, procedureType, languageCode, fileNameJson, fileNameQrd, fileNameLog, fsMountName, localEnv)
    
    partitionedJsonPaths = [ path.split(pathSep)[-1] for path in partitionedJsonPaths]
    flowLogger.logFlowCheckpoint(str(partitionedJsonPaths))
    
    flowLogger.logFlowCheckpoint("Completed Json Split")
    
    flowLogger.logFlowCheckpoint("Started Processing Partitioned Jsons")
    
    for index, fileNamePartitioned in enumerate(partitionedJsonPaths):
        
        flowLogger.logFlowCheckpoint(f"\n\n\n\n||||||||||||||||||||||||||||||||{str(index)} ||||| {str(fileNamePartitioned)}||||||||||||||||||||||||||||||||\n\n\n\n")
        
        if index == 3:
            stopWordFilterLen = 100
            isPackageLeaflet = True
        else:
            stopWordFilterLen = 6
            isPackageLeaflet = False
            
        df, coll = extractAndValidateHeadings(domain,
                                    procedureType,
                                    languageCode,
                                    index,
                                    fileNamePartitioned,
                                    fileNameQrd,
                                    fileNameMatchRuleBook,
                                    fileNameDocumentTypeNames,
                                    fileNameLog,
                                    stopWordFilterLen=stopWordFilterLen,
                                    isPackageLeaflet=isPackageLeaflet,
                                    medName=medName,
                                    fsMountName=fsMountName,
                                    localEnv=localEnv)
        
        
        print(f"Completed Heading Extraction For File")
        flowLogger.logFlowCheckpoint("Completed Heading Extraction For File")
        
        print(f"Starting Document Annotation For File :- {fileNamePartitioned}")        
        flowLogger.logFlowCheckpoint("Starting Document Annotation For File")
        documentAnnotationObj = DocumentAnnotation(fileNamePartitioned,'c20835db4b1b4e108828a8537ff41506','https://spor-sit.azure-api.net/pms/api/v2/',df,coll)
        try:
            pms_oms_annotation_data = documentAnnotationObj.processRegulatedAuthorizationForDoc()
            print(pms_oms_annotation_data)
        except:
            pms_oms_annotation_data = None
            print("Error Found")
            
        print(f"Completed Document Annotation")        
        flowLogger.logFlowCheckpoint("Completed Document Annotation")
        
        print(f"Starting Extracting Content Between Heading For File :- {fileNamePartitioned}")        
        flowLogger.logFlowCheckpoint("Starting Extracting Content Between Heading")
        
        extractContentlogger =  MatchLogger(f'ExtractContentBetween_{index}_{getRandomString(1)}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)
        extractorObj = DataBetweenHeadingsExtractor(extractContentlogger, coll, domain, procedureType, languageCode, fsMountName, localEnv)
        dfExtractedHierRR = extractorObj.extractContentBetweenHeadings(fileNamePartitioned)
        
        print(f"Completed Extracting Content Between Heading")        
        flowLogger.logFlowCheckpoint("Completed Extracting Content Between Heading")
        
        xmlLogger =  MatchLogger(f'XmlGeneration_{index}_{getRandomString(1)}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)
        fhirXmlGeneratorObj = FhirXmlGenerator(xmlLogger, pms_oms_annotation_data, stylesFilePath, medName, fsMountName, localEnv)
        fileNameXml = fileNamePartitioned.replace('.json','.xml')
        generatedXml = fhirXmlGeneratorObj.generateXml(dfExtractedHierRR, fileNameXml)
        
        fhirServiceLogger =  MatchLogger(f'XML Submission Logger_{index}_{getRandomString(1)}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)

        fhirServiceObj = FhirService(fhirServiceLogger, generatedXml, fsMountName, localEnv)
        fhirServiceObj.submitFhirXml()
        print(f"Created XML File For :- {fileNamePartitioned}")        

        #return df,coll,dfExtractedHierRR
    
    flowLogger.logFlowCheckpoint("Completed Processing Partitioned Jsons")


C:\Users\vipsharm\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vipsharm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
parseDocument("F:\\Projects\\EMA\\Repository\\EMA EPI PoC\\function_code\\data\\converted_to_html\\H\\CAP\\en\\Adakveo-4874 EN PI.html", fileNameQrd, fileNameMatchRuleBook, fileNameDocumentTypeNames, medName = "Adakveo", fsMountName = '/mounted', localEnv= True)

2021-05-12 02:04:57,328 : Flow Logger HTML_Y : Starting HTML Conversion To Json | H | CAP |  en | HTML | Adakveo-4874 EN PI.html
2021-05-12 02:04:57,336 : Style Dictionary_A : Reading style dictionary in file: rule_dictionary_en.json | H | CAP |  en | HTML | Adakveo-4874 EN PI.html
2021-05-12 02:04:57,383 : Style Dictionary_A : Qrd Section Keys Retrieved For Style Dictionary: ANNEX I, ANNEX II, ANNEX III, B. PACKAGE LEAFLET | H | CAP |  en | HTML | Adakveo-4874 EN PI.html


['F:', 'Projects', 'EMA', 'Repository', 'EMA EPI PoC', 'function_code', 'data', 'converted_to_html', 'H', 'CAP', 'en', 'Adakveo-4874 EN PI.html']
Adakveo-4874 EN PI.html en CAP
------------- F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\H\CAP\en\Adakveo-4874 EN PI.txt -----------------
F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\converted_to_html\H\CAP\en\Adakveo-4874 EN PI.html F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\H\CAP\en\Adakveo-4874 EN PI.json


2021-05-12 02:04:57,839 : Parser_k : Style Information Stored In File: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\H\CAP\en\Adakveo-4874 EN PI.txt | H | CAP |  en | HTML | Adakveo-4874 EN PI.html
2021-05-12 02:04:59,222 : Parser_k : Writing to file: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\H\CAP\en\Adakveo-4874 EN PI.json | H | CAP |  en | HTML | Adakveo-4874 EN PI.html
2021-05-12 02:04:59,324 : Flow Logger HTML_Y : Completed HTML Conversion To Json | H | CAP |  en | HTML | Adakveo-4874 EN PI.html
2021-05-12 02:04:59,325 : Flow Logger HTML_Y : Starting Json Split | H | CAP |  en | HTML | Adakveo-4874 EN PI.html
2021-05-12 02:04:59,329 : Style Dictionary_M : Reading style dictionary in file: rule_dictionary_en.json | H | CAP |  en | Json | Adakveo-4874 EN PI.json
2021-05-12 02:04:59,377 : Style Dictionary_M : Qrd Section Keys Retrieved For Style Dictionary: ANNEX I, ANNEX II, ANNEX III, B. PACKAGE LEAFLET | H | CAP |  en | Json | Adak

F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\H\CAP\en\Adakveo-4874 EN PI.txt


2021-05-12 02:04:59,696 : Partition_i : Writing partition to file: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\en\Adakveo-4874 EN PI_SmPC.json | H | CAP |  en | Json | Adakveo-4874 EN PI.json
2021-05-12 02:04:59,725 : Partition_i : Writing partition to file: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\en\Adakveo-4874 EN PI_ANNEX II.json | H | CAP |  en | Json | Adakveo-4874 EN PI.json
2021-05-12 02:04:59,753 : Partition_i : Writing partition to file: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\en\Adakveo-4874 EN PI_ANNEX III.json | H | CAP |  en | Json | Adakveo-4874 EN PI.json
2021-05-12 02:04:59,763 : Partition_i : Writing partition to file: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\en\Adakveo-4874 EN PI_ PACKAGE LEAFLET.json | H | CAP |  en | Json | Adakveo-4874 EN PI.json
2021-05-12 02:04:59,773 : Flow Logger HTML_Y : ['Adakveo-4874 E

...

2021-05-12 02:05:09,354 : Heading Extraction Adakveo-4874 EN PI_SmPC.json_t : Match Failed | H | CAP |  en | 0 | Adakveo-4874 EN PI_SmPC.json | Doc txt :- 'Tabulated list of adverse reactions' | Qrd txt :- '6.6 Special precautions for disposal <and other handling>' | Matched :- 'False'


lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseChe

 reactions in clinical studies' | Qrd txt :- '6.6 Special precautions for disposal <and other handling>' | Matched :- 'False'


lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseCheckFuzzyScoreThreshhold 90
lowerCaseChe

KeyboardInterrupt: 

In [4]:
parseDocument("F:\\Projects\\EMA\\Repository\\EMA EPI PoC\\function_code\\data\\converted_to_html\\H\\CAP\\bg\\emea-combined-h-2494-bg.htm", fileNameQrd, fileNameMatchRuleBook, fileNameDocumentTypeNames, medName = "Kalydeco", fsMountName = '/mounted', localEnv= True)

2021-05-11 02:37:10,668 : Flow Logger HTML_T : Starting HTML Conversion To Json | H | CAP |  bg | HTML | emea-combined-h-2494-bg.htm
2021-05-11 02:37:10,687 : Style Dictionary_7 : Reading style dictionary in file: rule_dictionary_bg.json | H | CAP |  bg | HTML | emea-combined-h-2494-bg.htm
2021-05-11 02:37:10,790 : Style Dictionary_7 : Qrd Section Keys Retrieved For Style Dictionary: ПРИЛОЖЕНИЕ I, ПРИЛОЖЕНИЕ II, ПРИЛОЖЕНИЕ III, B.  ЛИСТОВКА | H | CAP |  bg | HTML | emea-combined-h-2494-bg.htm


['F:', 'Projects', 'EMA', 'Repository', 'EMA EPI PoC', 'function_code', 'data', 'converted_to_html', 'H', 'CAP', 'bg', 'emea-combined-h-2494-bg.htm']
emea-combined-h-2494-bg.htm bg CAP
------------- F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\H\CAP\bg\emea-combined-h-2494-bg.txt -----------------
F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\converted_to_html\H\CAP\bg\emea-combined-h-2494-bg.htm F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\H\CAP\bg\emea-combined-h-2494-bg.json


2021-05-11 02:37:15,282 : Parser_2 : Style Information Stored In File: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\H\CAP\bg\emea-combined-h-2494-bg.txt | H | CAP |  bg | HTML | emea-combined-h-2494-bg.htm
2021-05-11 02:37:32,258 : Parser_2 : Writing to file: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\outputJSON\H\CAP\bg\emea-combined-h-2494-bg.json | H | CAP |  bg | HTML | emea-combined-h-2494-bg.htm
2021-05-11 02:37:33,083 : Flow Logger HTML_T : Completed HTML Conversion To Json | H | CAP |  bg | HTML | emea-combined-h-2494-bg.htm
2021-05-11 02:37:33,086 : Flow Logger HTML_T : Starting Json Split | H | CAP |  bg | HTML | emea-combined-h-2494-bg.htm
2021-05-11 02:37:33,141 : Style Dictionary_X : Reading style dictionary in file: rule_dictionary_bg.json | H | CAP |  bg | Json | emea-combined-h-2494-bg.json
2021-05-11 02:37:33,254 : Style Dictionary_X : Qrd Section Keys Retrieved For Style Dictionary: ПРИЛОЖЕНИЕ I, ПРИЛОЖЕНИЕ II, ПРИЛОЖЕНИЕ III, B.

Starting Heading Extraction For File :- emea-combined-h-2494-bg_SmPC.json
File being processed: F:\Projects\EMA\Repository\EMA EPI PoC\function_code\data\partitionedJSONs\H\CAP\bg\emea-combined-h-2494-bg_SmPC.json
--------------------------------------------
SmPC


2021-05-11 02:37:35,419 : Heading Extraction emea-combined-h-2494-bg_SmPC.json : Started Extracting Heading | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json
 НА ПРОДУКТА' | Qrd txt :- 'КРАТКА ХАРАКТЕРИСТИКА НА ПРОДУКТА' | Matched :- 'True'atch Passed | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json | Doc txt :- 'КРАТКА
2021-05-11 02:37:35,942 : Heading Extraction emea-combined-h-2494-bg_SmPC.json : Validation Passed As This The First Heading | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json | currHeadId :- '1' | currParentHeadId :- 'nan' | prevParentHeadId :- ''
 ПРОДУКТ' | Qrd txt :- '1. ИМЕ НА ЛЕКАРСТВЕНИЯ ПРОДУКТ' | Matched :- 'True'son : Match Passed | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json | Doc txt :- '1.       ИМЕ
2021-05-11 02:37:36,003 : Heading Extraction emea-combined-h-2494-bg_SmPC.json : Validation Flow Is Broken | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json | currHeadId :- '3' | currParentHeadId :- '1.0' | prevParentHeadId :- '

...

----------------------------------
RemovedByStyle
----------------------------------


 безопасност' | Qrd txt :- 'Клинична ефикасност и безопасност' | Matched :- 'True'atch Passed | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json | Doc txt :- 'Клинична
2021-05-11 02:38:18,707 : Heading Extraction emea-combined-h-2494-bg_SmPC.json : Validation Flow Is Broken | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json | currHeadId :- '36' | currParentHeadId :- '33.0' | prevParentHeadId :- '1'
2021-05-11 02:38:18,718 : Heading Extraction emea-combined-h-2494-bg_SmPC.json : Validation Passed | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json | currHeadId :- '36' | currParentHeadId :- '33.0' | prevParentHeadId :- '1'
2021-05-11 02:38:18,724 : Heading Extraction emea-combined-h-2494-bg_SmPC.json : Validation Failed By Style | H | CAP |  bg | 0 | emea-combined-h-2494-bg_SmPC.json | currHeadId :- '36' | currParentHeadId :- '33.0' | prevParentHeadId :- '1'


----------------------------------
RemovedByStyle
----------------------------------


 каналопатии ' | Qrd txt :- '6.5 Вид и съдържание на опаковката <и специални приспособления за употреба, приложение или имплантиране>' | Matched :- 'False' txt :- 'Проучвания 1
 лечение. ' | Qrd txt :- '6.5 Вид и съдържание на опаковката <и специални приспособления за употреба, приложение или имплантиране>' | Matched :- 'False'Doc txt :- 'Първичната
 са дадени в Таблица 6.' | Qrd txt :- '6.5 Вид и съдържание на опаковката <и специални приспособления за употреба, приложение или имплантиране>' | Matched :- 'False'езултатите
 проучванията1 и2' | Qrd txt :- '6.5 Вид и съдържание на опаковката <и специални приспособления за употреба, приложение или имплантиране>' | Matched :- 'False' :- 'Таблица 6:
 събитията' | Qrd txt :- '6.5 Вид и съдържание на опаковката <и специални приспособления за употреба, приложение или имплантиране>' | Matched :- 'False'Doc txt :- 'ДИ:
 събитията' | Qrd txt :- '6.6 Специални предпазни мерки при изхвърляне <и работа>' | Matched :- 'False' |  bg | 0 | emea-combined

KeyboardInterrupt: 